In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('housing.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [37]:
df['total_bedrooms'] = df['total_bedrooms'].fillna(df['total_bedrooms'].mean())

In [38]:
from sklearn.model_selection import train_test_split  #Datani train va testga bo‘lish uchun.
from sklearn.preprocessing import StandardScaler, OneHotEncoder #Sonli ustunlarni scale qilish uchun (StandardScaler),Kategoriyani raqamga aylantirish uchun (OneHotEncoder)
from sklearn.ensemble import RandomForestRegressor  #  ML algoritm — Random Forest Regression
from sklearn.metrics import mean_squared_error # Model xatosini hisoblaydi.
from sklearn.compose import ColumnTransformer # Numerical va categorical ustunlarni turlicha transformatsiya qilish uchun.
from sklearn.pipeline import make_pipeline    # Hammasini birga ulab, bitta “pipeline” yaratadi. 
import joblib  #  Modelni saqlash va yuklash uchun.

In [39]:
x = df.drop('median_house_value', axis=1)  # Feature  x — barcha belgilari (input features)

In [40]:
y = df['median_house_value'] # Target  y — biz bashorat qilmoqchi bo‘lgan narsa (target)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2) #80% — model o‘qitish uchun
#➡️ 20% — test qilish uchun

In [42]:
sc = StandardScaler() # sc = sonli ustunlarda scaling (normalizatsiya)
oe = OneHotEncoder()  # oe = kategoriyani raqamga aylantirish

In [43]:
numerical = x_train.select_dtypes(include='float').columns # Sonli ustunlarni avtomatik topadi 

In [44]:
categorical = x_train.select_dtypes(include='object').columns  # Matn ustunlarni avtomatik topadi

In [45]:
protcessor = ColumnTransformer(       # sonli ustunlar StandardScaler orqali o‘tadi
    transformers = [                  # kategoriy usyunlar OneHotEncoder orqali o‘tadi
        ('num', sc, numerical),       # ColumnTransformer nima qiladi? ML modelga tayyorlangan ko‘rinishdagi X ni beradi.
        ('cat', oe, categorical)      #
    ]                                 #
)                                     #

In [46]:
model = make_pipeline(protcessor, RandomForestRegressor()) #pipeline hamma bosqichlarni bitta tizimga ulash.

In [47]:
model.fit(x_train, y_train)  # modelni o'qitish.

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['ocean_proximity'], dtype='object'))])),
                ('randomforestregressor', RandomForestRegressor())])

In [48]:
preds = model.predict(x_test) # Test datasiga qarab uy narxlarini bashorat qiladi.

In [49]:
mse = mean_squared_error(y_test, preds) # Xato hisoblash

In [50]:
import numpy as np

In [51]:
print('MSE:', np.sqrt(mse)) ## MSE — o‘rtacha kvadrat xato, Katta bo‘lsa → yomon
                                        # Kichik bo‘lsa → yaxshi

MSE: 48053.38018008304


In [52]:
joblib.dump(model, 'house_pricing_forset.joblib') #modelni saqlash,faylga yozadi,keyinchalik qayta o'qimiz.

['house_pricing_forset.joblib']

In [53]:
load_model = joblib.load('house_pricing_forset.joblib')  # saqlangan modelni yuklash

In [54]:
test_household_data = pd.DataFrame({
    'longitude': [-118.45],
    'latitude': [34.05],               
    'housing_median_age': [25.0],
    'total_rooms': [3500.0],
    'total_bedrooms': [650.0],
    'population': [1800.0],
    'households': [600.0],
    'median_income': [5.5],
    'ocean_proximity': ['<1H OCEAN']
})                                     # yangi uyni test qilish.
                                       # Model:scaling qiladi,
                                       # encoding qiladi, random forest bilan baho beradi,ya’ni uy narxini bashorat qiladi.